In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import re
import pickle

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rc
rc('text', usetex=True)

def bold_text(string):
    return r'\textbf{{{}}}'.format(string)

from IPython.display import Markdown
def printmd(string):
    """Embed the input string into Markdown."""
    display(Markdown(string))
    
def list_files(startpath):
    level_colours = {0: '#339fff', 1: '#ff5b33'}
    
    for root, dirs, files in os.walk(startpath):
        if os.path.basename(root) == startpath:
            continue
        
        level = root.replace(startpath, '').count(os.sep) - 1        
        indent = ' ' * 4 * (level)
        
        printmd('<pre>{}<b><font color={}>{}</font></b></pre>'.format(indent, level_colours[level], os.path.basename(root)))
        
        if len(files) > 0:
            print('{}{}'.format(indent, files))

# Importing data

Explore the contents of the folder with all data files

In [2]:
data_folder = 'session_210302'

In [3]:
printmd('**Data contents**')
list_files(data_folder)

**Data contents**

<pre><b><font color=#339fff>Externalities</font></b></pre>

<pre>    <b><font color=#ff5b33>bystanders_negative</font></b></pre>

    ['batch172_deals.csv', 'batch172_games.csv', 'batch172_offers.csv', 'batch172_players.csv']


<pre>    <b><font color=#ff5b33>bystanders_positive</font></b></pre>

...

<pre><b><font color=#339fff>LimitedAsks</font></b></pre>

<pre>    <b><font color=#ff5b33>black_box</font></b></pre>

    ['batch168_deals.csv', 'batch168_games.csv', 'batch168_offers.csv', 'batch168_players.csv']


<pre>    <b><font color=#ff5b33>open_book</font></b></pre>

    ['batch169_deals.csv', 'batch169_games.csv', 'batch169_offers.csv', 'batch169_players.csv']


Store all data in the form ```{(market, treatment): {'deals': df_deals, 'games': df_games, 'offers': df_offers, 'players': df_players}}```

In [4]:
all_data = {}
data_types = []

for path, folders, files in os.walk(data_folder):
    for file in files:
        treatment = tuple(path.split('\\')[1:])
        dtype = re.match(r'^.*_(.*)\.csv.*$', file).group(1)
        data_types.append(dtype)
        
        if treatment not in all_data.keys():
            all_data[treatment] = {}
            
        all_data[treatment][dtype] = pd.read_csv('{}\\{}'.format(path, file))
        
data_types = set(data_types)

Check whether all .csv files share the same structure and print out the names of their columns

In [5]:
for dtype in data_types:
    printmd('**{}**'.format(dtype))
    data = [d[dtype] for d in all_data.values()]
    
    all([(data[0].columns.intersection(df.columns) == data[0].columns).all() for df in data])
    
    data[0].columns.to_list()

**games**

True

['game_id',
 'game_iid',
 'batch_id',
 'status',
 'title',
 'keeps',
 'randomizes',
 'best',
 'full_info',
 'no_info',
 'other_info',
 'own_info',
 'elapsed_time']

**players**

True

...

True

['price',
 'bprice',
 'bid_id',
 'buyer_id',
 'buyer_iid',
 'sprice',
 'ask_id',
 'seller_id',
 'seller_iid',
 'round_id',
 'round_iid',
 'game_id',
 'game_iid',
 'time',
 'randomized',
 'keeps']

**offers**

True

['amount',
 'time',
 'game_id',
 'game_iid',
 'round_id',
 'round_iid',
 'player_id',
 'side',
 'status',
 'type',
 'matched_price',
 'offer_db_id']

Note:\
```var_id``` global id\
```var_iid``` local id

## Game information

In [6]:
all_data[('Externalities', 'bystanders_negative')]['games'].columns.to_list()

['game_id',
 'game_iid',
 'batch_id',
 'status',
 'title',
 'keeps',
 'randomizes',
 'best',
 'full_info',
 'no_info',
 'other_info',
 'own_info',
 'elapsed_time']

Find all columns with non-constant values

In [7]:
for treatment, data in all_data.items():
    print(treatment, list(data['games'].columns[data['games'].nunique() > 1]))

('Externalities', 'bystanders_negative') ['game_id', 'game_iid', 'elapsed_time']
('Externalities', 'bystanders_positive') ['game_id', 'game_iid', 'elapsed_time']
('Externalities', 'normal') ['game_id', 'game_iid', 'elapsed_time']
('LimitedAsks', 'black_box') ['game_id', 'game_iid', 'elapsed_time']
('LimitedAsks', 'open_book') ['game_id', 'game_iid', 'elapsed_time']


In [8]:
for treatment, data in all_data.items():
    printmd('**{}**'.format(treatment))
    data['games'][['game_iid', 'title', 'elapsed_time']]

**('Externalities', 'bystanders_negative')**

,game_iid,title,elapsed_time
0,1,Full information / Best deal matching,1406
1,2,Full information / Best deal matching,1388
2,3,Full information / Best deal matching,1428
3,4,Full information / Best deal matching,1430
4,5,Full information / Best deal matching,1428


**('Externalities', 'bystanders_positive')**

,game_iid,title,elapsed_time
0,1,Full information / Best deal matching,1396
1,2,Full information / Best deal matching,1396
2,3,Full information / Best deal matching,1358
3,4,Full information / Best deal matching,1406
4,5,Full information / Best deal matching,1410
5,6,Full information / Best deal matching,1336


**('Externalities', 'normal')**

,game_iid,title,elapsed_time
0,1,Full information / Best deal matching,1412
1,2,Full information / Best deal matching,1392
2,3,Full information / Best deal matching,1367
3,4,Full information / Best deal matching,1415
4,5,Full information / Best deal matching,1390


**('LimitedAsks', 'black_box')**

,game_iid,title,elapsed_time
0,1,No information / Best deal matching,1300
1,2,No information / Best deal matching,1300
2,3,No information / Best deal matching,1300
3,4,No information / Best deal matching,1320
4,5,No information / Best deal matching,1315


**('LimitedAsks', 'open_book')**

,game_iid,title,elapsed_time
0,1,Full information / Best deal matching,1363
1,2,Full information / Best deal matching,1440
2,3,Full information / Best deal matching,1380
3,4,Full information / Best deal matching,1446
4,5,Full information / Best deal matching,1388


## Player information

In [9]:
all_data[('Externalities', 'bystanders_negative')]['players'].columns.to_list()

['player_id',
 'player_iid',
 'game_id',
 'game_iid',
 'rprice',
 'side',
 'iddle',
 'total_payoff']

Find all columns with non-constant values

In [10]:
for treatment, data in all_data.items():
    print(treatment, list(data['players'].columns[data['players'].nunique() > 1]))

('Externalities', 'bystanders_negative') ['player_id', 'player_iid', 'game_id', 'game_iid', 'rprice', 'side', 'iddle', 'total_payoff']
('Externalities', 'bystanders_positive') ['player_id', 'player_iid', 'game_id', 'game_iid', 'rprice', 'side', 'iddle', 'total_payoff']
('Externalities', 'normal') ['player_id', 'player_iid', 'game_id', 'game_iid', 'rprice', 'side', 'iddle', 'total_payoff']
('LimitedAsks', 'black_box') ['player_id', 'player_iid', 'game_id', 'game_iid', 'rprice', 'side', 'iddle', 'total_payoff']
('LimitedAsks', 'open_book') ['player_id', 'player_iid', 'game_id', 'game_iid', 'rprice', 'side', 'iddle', 'total_payoff']


## Offer information

In [11]:
all_data[('Externalities', 'bystanders_negative')]['offers'].columns.to_list()

['amount',
 'time',
 'game_id',
 'game_iid',
 'round_id',
 'round_iid',
 'player_id',
 'side',
 'status',
 'type',
 'matched_price',
 'offer_db_id']

In [12]:
for treatment, data in all_data.items():
    printmd('**{}**'.format(treatment))
    data_offers = data['offers']
    print('status: {}'.format(set(data_offers['status'])))
    print('type: {}'.format(set(data_offers['type'])))
    
    printmd('status == ```Accepted``` if and only if the bid/ask resulted in a deal')
    set(data_offers[data_offers['status'] == 'Replaced']['matched_price'].dropna())
    set(data_offers[data_offers['status'] == 'Expired']['matched_price'].dropna())
    set(data_offers[data_offers['matched_price'].notna()]['status'])
    
    printmd('type == ```Auto``` corresponds to accepting a deal')
    data_offers[(data_offers['type'] == 'Auto') & (data_offers['matched_price'].isna())]

**('Externalities', 'bystanders_negative')**

status: {'Accepted', 'Expired', 'Replaced'}
type: {'Manual', 'Auto'}


status == ```Accepted``` if and only if the bid/ask resulted in a deal

set()

set()

...

set()

set()

{'Accepted'}

type == ```Auto``` corresponds to accepting a deal

,amount,time,game_id,game_iid,round_id,round_iid,player_id,side,status,type,matched_price,offer_db_id


Add treatments information and remove redundant/unnecessary columns

In [13]:
all_data.keys()

dict_keys([('Externalities', 'bystanders_negative'), ('Externalities', 'bystanders_positive'), ('Externalities', 'normal'), ('LimitedAsks', 'black_box'), ('LimitedAsks', 'open_book')])

In [14]:
treatment_names = {
    ('Externalities', 'bystanders_negative'): 'FullExtNeg',
    ('Externalities', 'bystanders_positive'): 'FullExtPos',
    ('Externalities', 'normal'): 'FullExtNorm',
    ('LimitedAsks', 'black_box'): 'BBLimS',
    ('LimitedAsks', 'open_book'): 'FullLimS'
}

In [15]:
for treatment, data in all_data.items():
    #data['offers'].drop(['game_id', 'round_id', 'status'], axis=1, inplace=True)
    # Keep the status column
    data['offers'].drop(['game_id', 'round_id'], axis=1, inplace=True)
    data['offers']['treatment'] = treatment_names[treatment]
    data['offers'].rename({'game_iid': 'game', 'round_iid': 'round', 'amount': 'bid',
                           'player_id': 'id', 'matched_price': 'price'}, axis=1, inplace=True)

Add ```match_id``` and ```match_time```

In [16]:
for treatment, data in all_data.items():
    for idx, row in data['deals'].iterrows():
        game, rnd, match_time, buyer, seller, askID, bidID, bprice, sprice = row[['game_iid', 'round_iid', 'time', 'buyer_id',
                                                                                  'seller_id', 'ask_id', 'bid_id', 'bprice', 'sprice']]
        
        game_round = (data['offers']['game'] == game) & (data['offers']['round'] == rnd)
        ask_row = (data['offers']['offer_db_id'] == askID)
        bid_row = (data['offers']['offer_db_id'] == bidID)
        
        data['offers'].loc[game_round & ask_row, 'match_time'] = match_time
        data['offers'].loc[game_round & ask_row, 'match_id'] = buyer
        data['offers'].loc[game_round & ask_row, 'price_temp'] = sprice
        
        data['offers'].loc[game_round & bid_row, 'match_time'] = match_time
        data['offers'].loc[game_round & bid_row, 'match_id'] = seller
        data['offers'].loc[game_round & bid_row, 'price_temp'] = bprice

In [17]:
for treatment, data in all_data.items():
    data['offers']['price'].equals(data['offers']['price_temp'])

True

True

True

True

True

In [18]:
for treatment, data in all_data.items():
    data['offers'].drop(['price_temp'], axis=1, inplace=True)

Add ```valuation```

In [19]:
for treatment, data in all_data.items():
    for (game, idx), dfi in data['offers'].groupby(['game', 'id']):
        val = data['players'][data['players']['player_id']  == idx]['rprice'].values[0]
        data['offers'].loc[dfi.index, 'valuation'] = val

Rearrange to match the order in the rest of the data

In [20]:
for treatment, data in all_data.items():
    data['offers'] = data['offers'][['treatment', 'game', 'round', 'time', 'id', 'side', 'valuation',
                                     'bid', 'price', 'match_id', 'match_time', 'type', 'status']]

# Merging data

Store all datasets in a single dataframe

In [21]:
df = pd.DataFrame()

for treatment, data in all_data.items():
    df = df.append(data['offers'], ignore_index=True)

Create globally unique subject IDs

In [22]:
# Create globally unique subject IDs
df['old_id'] = df['id']
df['id'] = df.groupby(['treatment', 'game', 'id']).ngroup()

# Update the column with match IDs accordingly
for (treatment, game), df_game in df.groupby(['treatment', 'game']):
    for idx, row in df_game[df_game['match_id'].notna()].iterrows():
        df.loc[idx, 'match_id'] = df_game[df_game['old_id'] == row['match_id']]['id'].iloc[0]

df.drop(columns=['old_id'], axis=1, inplace=True)

Cast the valuations to ```int```

In [23]:
(df['valuation'] % 1 == 0).all()

True

In [24]:
df['valuation'] = df['valuation'].astype(int)

When a buyer and a seller are automatically matched under the first-price mechanism, a new entry with the bid/ask equal to the resulting price is automatically generated for the buyer/seller who submitted the bid/ask last. Remove all such entries and copy the corresopnding prices to the entries with the bids/asks submitted last.

In [25]:
df[['type', 'status']].drop_duplicates()
df.groupby(['type', 'status']).size()

,type,status
0,Manual,Expired
2,Manual,Accepted
3,Manual,Replaced
4,Auto,Accepted


type    status  
Auto    Accepted    1178
Manual  Accepted    1178
        Expired     7148
        Replaced    4821
dtype: int64

The status of type ```Auto``` can only be ```Accepted```

In [26]:
set(df[df['type'] == 'Auto']['status'])

{'Accepted'}

The status of a bid/ask is set to ```Accepted``` if and only if it results in a deal

In [27]:
set(df[df['price'].notna()]['status'])
df[df['status'] == 'Accepted']['price'].isna().any()

{'Accepted'}

False

Each bid–ask pair striking a deal is stored as follows: the first of the two is recorded as ``Manual``, the second as ``Auto``.

In [28]:
df_prices = df[df['price'].notna()]

In [29]:
bid_ask_pairs = {'MM': 0, 'MA': 0, 'AA': 0}

for (treatment, game, rnd), dfr in df_prices.groupby(['treatment', 'game', 'round']):
    for row_id, row in dfr.iterrows():
        if row['id'] < row['match_id']:
            id1 = row['id']
            id2 = row['match_id']
            
            types = {dfr[dfr['id'] == id1]['type'].iloc[0], dfr[dfr['id'] == id2]['type'].iloc[0]}

            if len(types) == 2:
                bid_ask_pairs['MA'] += 1
            elif types == {'Manual'}:
                bid_ask_pairs['MM'] += 1
            else:
                bid_ask_pairs['AA'] += 1

bid_ask_pairs

{'MM': 0, 'MA': 1178, 'AA': 0}

```Auto``` always take place after ```Manual``` (or, possibly, simultaneously)

A match is made at most 1 second after a bid and an ask are compatible

In [30]:
times = {'same': 0, 'M then A': 0, 'A then M': 0}
indices = {'M then A': 0, 'A then M': 0}
delays_to_match = []

for (treatment, game, rnd), dfr in df_prices.groupby(['treatment', 'game', 'round']):
    for row_id, row in dfr.iterrows():
        if row['id'] < row['match_id']:
            match = dfr[dfr['id'].isin([row['id'], row['match_id']])]
            
            types = set(match['type'])

            if len(types) == 2:
                M_time = match[match['type'] == 'Manual']['time'].iloc[0]
                A_time = match[match['type'] == 'Auto']['time'].iloc[0]

                M_id = match[match['type'] == 'Manual'].index
                A_id = match[match['type'] == 'Auto'].index

                if M_time == A_time:
                    times['same'] += 1
                elif M_time < A_time:
                    times['M then A'] += 1
                else:
                    times['A then M'] += 1

                if M_id < A_id:
                    indices['M then A'] += 1
                else:
                    indices['A then M'] += 1

            if int(match['match_time'].iloc[0]) != max(match['time']):
                delays_to_match.append(int(match['match_time'].iloc[0]) - max(match['time']))
    
times
indices
delays_to_match

{'same': 64, 'M then A': 1114, 'A then M': 0}

{'M then A': 1178, 'A then M': 0}

[1, 1, 1]

<font color=blue>The redundant rows (automatic matching enforced by the computer) correspond to ```Auto``` bids/asks following ```Replaced``` bids/asks which were high/low enough to result in a deal</font>

In [31]:
df_new = df.copy()

In [32]:
df_new['redundant'] = False

status = {'Accepted': 0, 'Replaced': 0, 'Expired': 0}

for (treatment, game, rnd, idx), dfi in df_new.groupby(['treatment', 'game', 'round', 'id']):
    for row_id, row in dfi.iterrows():
        if row['type'] == 'Auto':
            if len(dfi) > 1:
                preceding = dfi.loc[:row.name].iloc[-2]

                status[preceding['status']] += 1

                if preceding['status'] == 'Replaced':
                    if row['side'] == 'Buyer':
                        if preceding['bid'] >= row['bid']:
                            df_new.loc[row.name, 'redundant'] = True
                            df_new.loc[preceding.name, 'price'] = row['price']
                            df_new.loc[preceding.name, 'match_id'] = row['match_id']
                            df_new.loc[preceding.name, 'match_time'] = row['match_time']
                    else:
                        if preceding['bid'] <= row['bid']:
                            df_new.loc[row.name, 'redundant'] = True
                            df_new.loc[preceding.name, 'price'] = row['price']
                            df_new.loc[preceding.name, 'match_id'] = row['match_id']
                            df_new.loc[preceding.name, 'match_time'] = row['match_time']

status

{'Accepted': 0, 'Replaced': 676, 'Expired': 234}

In [33]:
len(df_new)
len(df)

df_new.drop(['redundant', 'price', 'match_id', 'match_time'], axis=1).equals(df.drop(['price', 'match_id', 'match_time'], axis=1))

14325

14325

True

In [34]:
df_new = df_new[~df_new['redundant']]
df_new.drop('redundant', axis=1, inplace=True)

In [35]:
len(df_new)
df_new.groupby('type').size()

14024

type
Auto        877
Manual    13147
dtype: int64

In [36]:
df_prices = df_new[df_new['price'].notna()]

delays_to_match = []

for (treatment, game, rnd), dfr in df_prices.groupby(['treatment', 'game', 'round']):
    for row_id, row in dfr.iterrows():
        if row['id'] < row['match_id']:
            match = dfr[dfr['id'].isin([row['id'], row['match_id']])]
            
            if (len(match) != 2) or (match['match_time'].count() != 2) or (match['match_id'].count() != 2) or (match['price'].count() != 2):
                'Some data is missing'
            
            if int(match['match_time'].iloc[0]) != max(match['time']):
                delays_to_match.append(int(match['match_time'].iloc[0]) - max(match['time']))

delays_to_match

[1, 1, 1, 1, 1, 1]

In [37]:
for treatment, df_treatment in df.groupby(['treatment']):
    printmd(treatment)
    
    diff = pd.merge(df, df_new, how='outer', suffixes=('','_y'), indicator=True)
    diff = diff[diff['_merge'] != 'both']
    
    diff.sort_values(['treatment', 'game', 'round', 'time', 'id']).iloc[1:51]

BBLimS

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status,_merge
9214,BBLimS,1,1,5,3,Buyer,128,99,99.0,4.0,5.0,Auto,Accepted,left_only
14419,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced,right_only
9216,BBLimS,1,1,6,0,Seller,63,75,NaN,NaN,NaN,Manual,Replaced,left_only
9217,BBLimS,1,1,6,0,Seller,63,90,90.0,7.0,6.0,Auto,Accepted,left_only
14420,BBLimS,1,1,6,0,Seller,63,75,90.0,7.0,6.0,Manual,Replaced,right_only
9268,BBLimS,1,1,118,6,Buyer,118,100,NaN,NaN,NaN,Manual,Replaced,left_only
9269,BBLimS,1,1,118,6,Buyer,118,86,86.0,5.0,118.0,Auto,Accepted,left_only
14421,BBLimS,1,1,118,6,Buyer,118,100,86.0,5.0,118.0,Manual,Replaced,right_only
9273,BBLimS,1,2,4,3,Buyer,128,105,NaN,NaN,NaN,Manual,Replaced,left_only
9274,BBLimS,1,2,4,3,Buyer,128,85,85.0,0.0,4.0,Auto,Accepted,left_only


FullExtNeg

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status,_merge
9214,BBLimS,1,1,5,3,Buyer,128,99,99.0,4.0,5.0,Auto,Accepted,left_only
14419,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced,right_only
9216,BBLimS,1,1,6,0,Seller,63,75,NaN,NaN,NaN,Manual,Replaced,left_only
9217,BBLimS,1,1,6,0,Seller,63,90,90.0,7.0,6.0,Auto,Accepted,left_only
14420,BBLimS,1,1,6,0,Seller,63,75,90.0,7.0,6.0,Manual,Replaced,right_only
9268,BBLimS,1,1,118,6,Buyer,118,100,NaN,NaN,NaN,Manual,Replaced,left_only
9269,BBLimS,1,1,118,6,Buyer,118,86,86.0,5.0,118.0,Auto,Accepted,left_only
14421,BBLimS,1,1,118,6,Buyer,118,100,86.0,5.0,118.0,Manual,Replaced,right_only
9273,BBLimS,1,2,4,3,Buyer,128,105,NaN,NaN,NaN,Manual,Replaced,left_only
9274,BBLimS,1,2,4,3,Buyer,128,85,85.0,0.0,4.0,Auto,Accepted,left_only


FullExtNorm

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status,_merge
9214,BBLimS,1,1,5,3,Buyer,128,99,99.0,4.0,5.0,Auto,Accepted,left_only
14419,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced,right_only
9216,BBLimS,1,1,6,0,Seller,63,75,NaN,NaN,NaN,Manual,Replaced,left_only
9217,BBLimS,1,1,6,0,Seller,63,90,90.0,7.0,6.0,Auto,Accepted,left_only
14420,BBLimS,1,1,6,0,Seller,63,75,90.0,7.0,6.0,Manual,Replaced,right_only
9268,BBLimS,1,1,118,6,Buyer,118,100,NaN,NaN,NaN,Manual,Replaced,left_only
9269,BBLimS,1,1,118,6,Buyer,118,86,86.0,5.0,118.0,Auto,Accepted,left_only
14421,BBLimS,1,1,118,6,Buyer,118,100,86.0,5.0,118.0,Manual,Replaced,right_only
9273,BBLimS,1,2,4,3,Buyer,128,105,NaN,NaN,NaN,Manual,Replaced,left_only
9274,BBLimS,1,2,4,3,Buyer,128,85,85.0,0.0,4.0,Auto,Accepted,left_only


FullExtPos

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status,_merge
9214,BBLimS,1,1,5,3,Buyer,128,99,99.0,4.0,5.0,Auto,Accepted,left_only
14419,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced,right_only
9216,BBLimS,1,1,6,0,Seller,63,75,NaN,NaN,NaN,Manual,Replaced,left_only
9217,BBLimS,1,1,6,0,Seller,63,90,90.0,7.0,6.0,Auto,Accepted,left_only
14420,BBLimS,1,1,6,0,Seller,63,75,90.0,7.0,6.0,Manual,Replaced,right_only
9268,BBLimS,1,1,118,6,Buyer,118,100,NaN,NaN,NaN,Manual,Replaced,left_only
9269,BBLimS,1,1,118,6,Buyer,118,86,86.0,5.0,118.0,Auto,Accepted,left_only
14421,BBLimS,1,1,118,6,Buyer,118,100,86.0,5.0,118.0,Manual,Replaced,right_only
9273,BBLimS,1,2,4,3,Buyer,128,105,NaN,NaN,NaN,Manual,Replaced,left_only
9274,BBLimS,1,2,4,3,Buyer,128,85,85.0,0.0,4.0,Auto,Accepted,left_only


FullLimS

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status,_merge
9214,BBLimS,1,1,5,3,Buyer,128,99,99.0,4.0,5.0,Auto,Accepted,left_only
14419,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced,right_only
9216,BBLimS,1,1,6,0,Seller,63,75,NaN,NaN,NaN,Manual,Replaced,left_only
9217,BBLimS,1,1,6,0,Seller,63,90,90.0,7.0,6.0,Auto,Accepted,left_only
14420,BBLimS,1,1,6,0,Seller,63,75,90.0,7.0,6.0,Manual,Replaced,right_only
9268,BBLimS,1,1,118,6,Buyer,118,100,NaN,NaN,NaN,Manual,Replaced,left_only
9269,BBLimS,1,1,118,6,Buyer,118,86,86.0,5.0,118.0,Auto,Accepted,left_only
14421,BBLimS,1,1,118,6,Buyer,118,100,86.0,5.0,118.0,Manual,Replaced,right_only
9273,BBLimS,1,2,4,3,Buyer,128,105,NaN,NaN,NaN,Manual,Replaced,left_only
9274,BBLimS,1,2,4,3,Buyer,128,85,85.0,0.0,4.0,Auto,Accepted,left_only


In [38]:
df = df_new.copy()

# Overview of the data

In [39]:
index = pd.MultiIndex.from_tuples(df[['treatment', 'game']].drop_duplicates().itertuples(index=False, name=None),
                                  names=['Treatment', 'Game'])
overview = pd.DataFrame(index=index, columns=['Buyers', 'Sellers', 'Bids', 'Asks'])

for (treatment, game, side), df_side in df.groupby(['treatment', 'game', 'side']):
    if side == 'Buyer':
        overview.loc[(treatment, game), 'Buyers'] = len(set(df_side['id']))
        overview.loc[(treatment, game), 'Bids'] = len(df_side)
    elif side == 'Seller':
        overview.loc[(treatment, game), 'Sellers'] = len(set(df_side['id']))
        overview.loc[(treatment, game), 'Asks'] = len(df_side)
    else:
        print('No side provided.')

In [40]:
overview

Buyers Sellers Bids Asks
Treatment   Game                         
FullExtNeg  1         6       8  238  443
            2         5       8  212  430
            3         7       9  264  530
            4         8       9  357  381
            5         9       7  246  275
FullExtPos  1         6       8  200  288
            2         5       7  113  289
            3         4       4   57  103
            4         8       7  404  272
            5         8       8  335  431
            6         2       3  111  164
FullExtNorm 1         7       7  386  289
            2         8       5  363  204
            3         6       5  204  100
            4         9       7  329  208
            5         5       8  279  610
BBLimS      1         6       6  255  292
            2         7       7  314  261
            3         6       5  278  307
            4        10       3  345   85
            5         7       9  219  284
FullLimS    1         8       5  251  132
            2         9       9  339  275
            3         5       8  127  209
            4         8       9  218  147
            5         8       6  309  262

# Exporting data

## Externalities

In [41]:
df_ext = df[df['treatment'].str.contains('Ext')].copy()

Create globally unique subject IDs

In [42]:
# Create globally unique subject IDs
df_ext['old_id'] = df_ext['id']
df_ext['id'] = df_ext.groupby(['treatment', 'game', 'id']).ngroup()

# Update the column with match IDs accordingly
for (treatment, game), df_game in df_ext.groupby(['treatment', 'game']):
    for idx, row in df_game[df_game['match_id'].notna()].iterrows():
        df_ext.loc[idx, 'match_id'] = df_game[df_game['old_id'] == row['match_id']]['id'].iloc[0]

df_ext.drop(columns=['old_id'], axis=1, inplace=True)

In [43]:
df_ext

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status
0,FullExtNeg,1,1,5,12,Seller,108,125,NaN,NaN,NaN,Manual,Expired
1,FullExtNeg,1,1,5,7,Buyer,113,1,NaN,NaN,NaN,Manual,Expired
2,FullExtNeg,1,1,6,9,Buyer,108,101,101.0,0.0,7.0,Manual,Accepted
3,FullExtNeg,1,1,7,0,Seller,68,95,101.0,9.0,7.0,Manual,Replaced
5,FullExtNeg,1,1,7,11,Seller,103,160,NaN,NaN,NaN,Manual,Expired
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,FullExtNorm,5,10,108,137,Buyer,118,97,NaN,NaN,NaN,Manual,Expired
9205,FullExtNorm,5,10,112,136,Seller,83,103,NaN,NaN,NaN,Manual,Expired
9206,FullExtNorm,5,10,120,142,Seller,103,105,NaN,NaN,NaN,Manual,Expired
9207,FullExtNorm,5,10,124,136,Seller,83,103,103.0,137.0,126.0,Manual,Accepted


In [44]:
df_ext.to_csv('../Data/data_externalities.csv', index=False)

## Restricted asks

In [45]:
df_LimS = df[df['treatment'].str.contains('LimS')].copy()

Create globally unique subject IDs

In [46]:
# Create globally unique subject IDs
df_LimS['old_id'] = df_LimS['id']
df_LimS['id'] = df_LimS.groupby(['treatment', 'game', 'id']).ngroup()

# Update the column with match IDs accordingly
for (treatment, game), df_game in df_LimS.groupby(['treatment', 'game']):
    for idx, row in df_game[df_game['match_id'].notna()].iterrows():
        df_LimS.loc[idx, 'match_id'] = df_game[df_game['old_id'] == row['match_id']]['id'].iloc[0]

df_LimS.drop(columns=['old_id'], axis=1, inplace=True)

In [47]:
df_LimS

,treatment,game,round,time,id,side,valuation,bid,price,match_id,match_time,type,status
9209,BBLimS,1,1,4,4,Seller,78,99,99.0,3.0,5.0,Manual,Accepted
9210,BBLimS,1,1,4,7,Buyer,113,90,90.0,0.0,6.0,Manual,Accepted
9211,BBLimS,1,1,4,6,Buyer,118,5,NaN,NaN,NaN,Manual,Expired
9212,BBLimS,1,1,4,2,Seller,73,145,NaN,NaN,NaN,Manual,Expired
9213,BBLimS,1,1,5,3,Buyer,128,115,99.0,4.0,5.0,Manual,Replaced
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14320,FullLimS,5,10,113,140,Seller,108,128,NaN,NaN,NaN,Manual,Replaced
14321,FullLimS,5,10,116,133,Buyer,123,116,NaN,NaN,NaN,Manual,Replaced
14322,FullLimS,5,10,121,140,Seller,108,122,NaN,NaN,NaN,Manual,Replaced
14323,FullLimS,5,10,123,133,Buyer,123,115,115.0,140.0,129.0,Manual,Accepted


In [48]:
df_LimS.to_csv('../Data/data_restricted_asks.csv', index=False)